In [1]:
import numpy as np
import glob
import os, os.path
import math
import pandas as pd
import psycopg2

In [2]:
df = pd.read_csv('amazon_reduced.csv')

In [3]:
def psqlConnect(dbname, user, pwd, host, port):
    conn = None
    try:
        conn = psycopg2.connect(
            dbname=dbname,
            user=user,
            password=pwd,
            host=host,
            port=port
        )
        print("Successfully connected to the database.")
    except psycopg2.Error as e:
        print(f"An error occurred while connecting to the database: {e}")
    return conn

def load_age():
    try:
        cur.execute("LOAD 'age';")
    except psycopg2.Error as e:
        print(f"An error occurred: {e}")
        conn.rollback()
    
    try:
        cur.execute("SET search_path TO ag_catalog;")
    except psycopg2.Error as e:
        print(f"An error occurred: {e}")
        conn.rollback()
        
def execute(query):
    try:
        cur.execute(query)
        conn.commit()
        rows = cur.fetchall()
        for row in rows:
            print(row)
    except psycopg2.Error as e:
        print(f"An error occurred: {e}")
        conn.rollback()

In [4]:
#replace parameters with yoursdbname = "demo" 
user = "s9rt1v" 
pwd = "1234" 
host = "localhost" 
port = "5430"
conn = psqlConnect(dbname, user, pwd, host, port)
cur = conn.cursor()
load_age()

Successfully connected to the database.


In [ ]:
#create graph
graph = 'amazon'
query =f"SELECT create_graph('{graph}');"
execute(query)

In [ ]:
#create nodes:Product
for i in range(100000):
    n_dict={}
    for column in ['asin', 'title', 'salesrank', 'total_reviews', 'downloaded_reviews', 'avg_rating']:
        if not pd.isna(df_reduced.loc[i][column]):
            n_dict[column] = df_reduced.loc[i][column]
        else:
            n_dict[column] = ''
    n_dict['title'] = n_dict['title'].replace("'", "")
    n_dict['title'] = n_dict['title'].replace("$", "S")
    n_dict['title'] = n_dict['title'].replace("\\", "")
    query = f"""SELECT * FROM cypher('amazon', $$ CREATE (n:Product {{ asin: '{n_dict['asin']}', title: '{n_dict['title']}', salesrank: '{n_dict['salesrank']}', total_reviews: '{n_dict['total_reviews']}', downloaded_reviews: '{n_dict['downloaded_reviews']}', avg_rating: '{n_dict['avg_rating']}' }}) $$) AS (a agtype);"""
    execute(query)

In [ ]:
#create nodes:Group
group = df['group'].unique().tolist()
for i in range(len(group)-1):
    name = group[i+1]
    query = f"""SELECT * FROM cypher('amazon', $$ CREATE (n:Group {{name: '{name}'}})$$) AS (a agtype);"""
    execute(query)

In [ ]:
#create edges:BELONGS_TO for node product and group
for i in range(df.shape[0]):
    if not pd.isna(df.iloc[i]['group']):
        group = df.iloc[i]['group']
        asin = df.iloc[i]['asin']
        query = f"SELECT * FROM cypher ('amazon', $$ MATCH (source:Product {{ asin: '{asin}' }}), (target:Group {{ name: '{group}' }}) CREATE (source)-[:BELONGS_TO]->(target) $$) as (a agtype);"
        execute(query)


In [ ]:
#create edges:SIMILAR_TO for node product and product
for i in range(df.shape[0]):
    if not pd.isna(df.iloc[i]['similar']):
        sim_asins =  df.iloc[i]['similar'].split(', ')
        asin = df.iloc[i]['asin']
        similarities = df.iloc[i]['similarity']
        for i in range(len(sim_asins)):
            sim_asin = sim_asins[i]
            similarity = int(similarities[i])
            query = f"SELECT * FROM cypher ('amazon', $$ MATCH (source:Product {{ asin: '{asin}' }}), (target:Product {{ asin: '{sim_asin}' }}) CREATE (source)-[:SIMILAR_TO{{similarity:{similarity}}}]->(target) $$) as (a agtype);"
            execute(query)